# 10. Generating cell segmentation masks for the SABER amplified tag samples
## Summary
This notebook generates nuclear and cell segmentation masks from the repacked hdf files generated by the `3_normalisation_and_repacking` notebook using Mesmer from the Deepcell package. 

The input hdf files contain the image stacks to segment as well as metadata about which channels to aggregate for nuclear and cell segmentation, and what pixel size to use for the segmentation model. 

The nuclei and cell masks generated by Mesmer are also matched to generate further masks with match cell and nuclei labels. Matched nuclear and cell masks (matched by the same integer id) enable nuclear-cell marker comparisons to be made in downstream analysis. 

Masks generated are written back to the 'masks' node of the input hdf file. Summary plots for generated masks are also made. 

In [1]:
import pathlib
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from shutil import copyfile

import sys
sys.path.insert(0, '..\\code\\')
import high_plex_hdf

Set input and output directories

In [2]:
# Set data directory to work from 
base_dir = "C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF"
base_dir = pathlib.Path(base_dir)
base_sub_dir = base_dir / 'data' / 'processed' / 'xrf'

# Identify analysis directory
analysis_dir = base_dir / 'data' / 'analysis' / 'xrf'

# Specify the input directory where hdf files to process are located
input_dir = base_sub_dir / '3_norm_repacked_XRF_hdfs'

# Gather filepaths for preprocessed hdfs and config files for XRF fitting
hdf_filepaths = list(input_dir.glob('*.h5'))

# Make output directory segmented images
out_dir = analysis_dir / '10_XRF_SABER_cell_segmentation'
out_dir.mkdir(parents=True, exist_ok=True)
print('Segmented hdf files will be output to: \n\t', out_dir) 

# Read in scans and original scan metadata
df_hdf_files = pd.read_csv(base_sub_dir / '1_reduced_reshaped_hdfs' / 'preprocessed_hdf_config_files.csv')
df_hdf_metadata = pd.read_csv(base_sub_dir / '1_reduced_reshaped_hdfs' / 'xrf_scan_metadata_full.csv')

Segmented hdf files will be output to: 
	 C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation


In [3]:
df_hdf_files.columns

Index(['hdf_file', 'config_file', 'step_um', 'dual_detector', 'detector'], dtype='object')

In [4]:
# Select files from GeCMOS (2nd) run for segmenting
hdfs_to_segment = df_hdf_metadata['hdf_filename'][
    (df_hdf_metadata['detector'] == 'GeCMOS (2nd)') &
    (df_hdf_metadata['sample_type'].isin(['cell pellet', 'TMA']))
].to_list()

hdfs_to_segment = [f'{hdf}_normalised' for hdf in hdfs_to_segment]

Specify hdf files names to be segmented and a label name for each

Copy hdfs to be segmented to output directory

In [5]:
for sample_name in hdfs_to_segment:
    source_fpath = list(input_dir.glob(f'{sample_name}.h5'))[0]
    output_fpath = out_dir / f'{source_fpath.stem}_segmented.h5'
    copyfile(source_fpath, output_fpath)    

Generate segmentation masks for hdf files in output directory and write these back to masks node

In [6]:
hdf_filepaths = list(out_dir.glob('*.h5'))

for hdf_fpath in hdf_filepaths:
    print(hdf_fpath)
    hdf_segment = high_plex_hdf.hdf_to_HighPlex_Hdf(hdf_fpath=hdf_fpath)
    hdf_segment.mesmer_segment()
    hdf_segment.cell_nuclei_match()
    hdf_segment.export_masks_to_source_hdf()

#     # Check output with these functions, can move into the hdf_filepaths loop
#     # Moved out to reduce filesize
#     hdf_segment.mesmer_output_plot(out_dir = out_dir)
#     hdf_segment.segmented_masks_plot(out_dir = out_dir)

C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\bl_cp_column1_10Hz_0001_normalised_segmented.h5
pixel size:  0.5


C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


matched_cell_mask 93
matched_nuclei_mask 93
matched_non_nuclei_mask 93
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\bl_cp_column1_250Hz_0001_normalised_segmented.h5
pixel size:  0.5


C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


matched_cell_mask 71
matched_nuclei_mask 71
matched_non_nuclei_mask 71
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\bl_cp_column1_50Hz_0001_normalised_segmented.h5
pixel size:  0.5


...

matched_cell_mask 1241
matched_nuclei_mask 1241
matched_non_nuclei_mask 1241
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\st_tt_lbh4_inset_10Hz_0001_normalised_segmented.h5
pixel size:  0.5


C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


matched_cell_mask 103
matched_nuclei_mask 103
matched_non_nuclei_mask 103
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\st_tt_lbh4_inset_50Hz_0001_normalised_segmented.h5
pixel size:  0.5


C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\MerrickS\Anaconda3\envs\mez-xrf\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


matched_cell_mask 103
matched_nuclei_mask 103
matched_non_nuclei_mask 103


In [7]:
# hdf_segment.sample_metadata

In [8]:
# path = r"C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\analysis\xrf\10_XRF_SABER_cell_segmentation\bl_tt_lbh4_50Hz_0001_normalised_segmented.h5"

In [9]:
# hdf_segment = high_plex_hdf.hdf_to_HighPlex_Hdf(hdf_fpath=path)
# hdf_segment.mesmer_segment()
# hdf_segment.mesmer_output_plot(out_dir = out_dir)
# hdf_segment.cell_nuclei_match()
# hdf_segment.segmented_masks_plot(out_dir = out_dir)

In [10]:
# hdf_segment.nuclear_channels

In [11]:
# plt.imshow(hdf_segment.get_channel_image("Rh_Ka"))

In [12]:
# for hdf_fpath in hdf_filepaths:
#     hdf_segment = high_plex_hdf.hdf_to_HighPlex_Hdf(hdf_fpath=hdf_fpath)
#     hdf_segment.mesmer_segment()
# #     hdf_segment.mesmer_output_plot(out_dir = out_dir)
#     hdf_segment.cell_nuclei_match()
# #     hdf_segment.segmented_masks_plot(out_dir = out_dir)
#     hdf_segment.export_masks_to_source_hdf()

In [13]:
# for hdf_fpath in hdf_filepaths:
#     print(hdf_fpath)

In [14]:
# filepaths_7c = [
#     r'C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF/data/analysis/xrf/10_XRF_SABER_cell_segmentation/bl_tt_h2_50Hz_0001_normalised_segmented.h5',
#     r'C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF/data/analysis/xrf/10_XRF_SABER_cell_segmentation/bl_tt_lb2_50Hz_0001_normalised_segmented.h5',
#     r'C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF/data/analysis/xrf/10_XRF_SABER_cell_segmentation/bl_tt_lbh4_50Hz_0001_normalised_segmented.h5',    
# ]

# for hdf_fpath in filepaths_7c[:1]:
#     hdf_segment = high_plex_hdf.hdf_to_HighPlex_Hdf(hdf_fpath=hdf_fpath)
#     hdf_segment.mesmer_segment_named_channels(nuclear_channels=['Ho_Ka', 'Rh_Ka'])
#     hdf_segment.mesmer_segment()
# #     hdf_segment.mesmer_output_plot(out_dir = out_dir)
#     hdf_segment.cell_nuclei_match()
# #     hdf_segment.segmented_masks_plot(out_dir = out_dir)
#     hdf_segment.export_masks_to_source_hdf()

In [15]:
# hdf_segment.nuclear_channels

In [16]:
# hdf_segment.channel_metadata['plot_channel'][hdf_segment.channel_metadata['plot_channel'].isin(['Ho_Ka', 'Rh_Ka'])]

In [17]:
# list(hdf_segment.channel_metadata['plot_channel'][hdf_segment.channel_metadata['plot_channel'].isin(['Ho_Ka', 'Rh_Ka'])].index)

In [18]:
# list(hdf_segment.nuclear_channels.index)